# TestScraper for integrating LinkedIn in the future

## Install the requirements

In [30]:
!pip3 install ipython 
!pip3 install selenium  
!pip3 install time 
!pip3 install parsel
!pip3 install csv

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


## Needed imports ##

In [25]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time



## Enter twitter access tokens ##

In [5]:
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

## function for scraping tweets

In [12]:
def scraptweets(search_words, date_since, numTweets, numRuns):
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    program_start = time.time()
    for i in range(0,numRuns):
        start_run = time.time()
        tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)
        tweet_list = [tweet for tweet in tweets]
        noTweets = 0
        for tweet in tweet_list:
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  
                text = tweet.full_text
                ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
                
                db_tweets.loc[len(db_tweets)] = ith_tweet
                noTweets+=1
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920)
        
    
    print(db_tweets.head())
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))
        
            

In [ ]:
search_words = "#hongkong OR #hkprotests OR #freehongkong OR #hongkongprotests OR #hkpolicebrutality OR #antichinazi OR #standwithhongkong OR #hkpolicestate OR #HKpoliceterrorist OR #standwithhk OR #hkpoliceterrorism"
date_since = "2019-11-03"
numTweets = 2500
numRuns = 1# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

## LinkedIn Scraper

In [66]:
from linkedin_scraper import Person, actions
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome('/home/dazedtiara6667/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver')

In [41]:
username = "rishab-mudliar-8022171b1"

In [62]:
email = ""
password = ""
actions.login(driver, email, password)


In [50]:
from linkedin_scraper import Person
person1 = Person('https://www.linkedin.com/in/%s'%username, driver = driver, scrape=False)
person1.scrape()


## Scraping user details after logging in to linkedin

In [59]:
print("Name: " + person1.name)
for experience in person1.experiences :
    print("Experience: "+ str(experience))
for education in person1.educations :
    print("Education: " + str(education))
for interest in person1.interests:
    print("Interest: "+ str(interest))
for accomplishment in person1.accomplishments:
    print("Accomplishment: "+ str(accomplishment))
print (person1.linkedin_url)
print(person1.company)

Name: Rishab Mudliar
Experience: Professional Basketball Player at National Basketball Association (NBA) from 2004 – to  for 17 yrs based at United States
Experience: Venture Partner at None from None to None for None based at None
Experience: Investment Committee at None from None to None for None based at None
Experience: Board Member at None from None to None for None based at None
Experience: Vice President at National Basketball Players Association (NBPA) from Feb 2013 to Present for 8 yrs 1 mo based at New York, NY
Experience: Professional Basketball Player at National Basketball Association (NBA) from 2004 – to  for 17 yrs based at United States
Experience: Venture Partner at None from None to None for None based at None
Experience: Investment Committee at None from None to None for None based at None
Experience: Board Member at None from None to None for None based at None
Experience: Vice President at National Basketball Players Association (NBPA) from Feb 2013 to Present for 

## Another way to scrape LinkedIn

In [59]:
from parsel import Selector
# xpath to extract the text from the class containing the name
sel = Selector(text=driver.page_source) 
name = sel.xpath('//*[starts-with(@class, "inline t-24 t-black t-normal break-words")]/text()').extract_first()

if name:
    name = name.strip()

job_title = sel.xpath('//*[starts-with(@class, "mt1 t-18 t-black t-normal break-words")]/text()').extract_first()

if job_title:
    job_title = job_title.strip()
    
company = sel.xpath('//*[starts-with(@class, "text-align-left ml2 t-14 t-black t-bold full-width lt-line-clamp lt-line-clamp--multi-line ember-view")]/text()').extract_first()

if company:
    company = company.strip()



# xpath to extract the text from the class containing the location
location = sel.xpath('//*[starts-with(@class, "t-16 t-black t-normal inline-block")]/text()').extract_first()

if location:
    location = location.strip()
    
linkedin_url = driver.current_url

print("Name: " + name)
print("Job Title: "+ job_title)
print("LinkedIn URL: "+ linkedin_url)
print("Company: " + company)
print("Location: "+ location)


Name: Rishab Mudliar
Job Title: Member at FOSS@Amrita | IoT | Problem Solving | Android(Amateur)
LinkedIn URL: https://www.linkedin.com/in/rishab-mudliar-8022171b1/
Company: FOSS@Amrita
Location: Nagpur, Maharashtra, India
[]
